# 모듈 2: 에이전트에 도구(Tools) 추가하기

이 모듈에서는 에이전트에 **도구(Tools)**를 추가하여 단순한 대화형 AI를 넘어, 외부 시스템과 상호작용하고 복잡한 작업을 수행할 수 있는 에이전트를 만드는 방법을 단계별로 알아본다. 도구는 에이전트의 기능을 확장하는 핵심 요소이다.

## 학습 목표

이 모듈을 완료하면 다음 내용을 이해할 수 있다.

- **ADK 도구의 개념**: 에이전트가 도구를 사용하는 원리를 이해한다.
- **내장 도구 사용**: Google 검색과 같은 ADK 내장 도구를 활용한다.
- **커스텀 도구 구현**: 나만의 Python 함수를 에이전트가 사용할 수 있는 도구로 변환한다.
- **멀티 도구 에이전트**: 여러 도구를 상황에 맞게 선택하여 사용하는 에이전트를 구현한다.

## 도구(Tools)란 무엇인가

ADK에서 **도구**는 에이전트가 세상을 인지하고 행동할 수 있게 해주는 '손과 발'과 같은 역할을 한다. LLM은 기본적으로 텍스트만 생성할 수 있지만, 도구를 통해 다음과 같은 작업이 가능해진다.

- **외부 정보 검색**: Google 검색, 뉴스 검색 등 실시간 정보 접근
- **데이터 조회**: 내부 데이터베이스나 API 호출
- **연산 수행**: 복잡한 수학 계산이나 데이터 처리
- **시스템 제어**: 파일 저장, 이메일 전송 등 실제 시스템 조작

에이전트는 사용자의 질문을 분석하고, 필요하다면 적절한 도구를 선택(Function Calling)하여 실행한 뒤 그 결과를 바탕으로 답변을 생성한다.

## 환경 설정

실습을 진행하기 위해 필요한 라이브러리를 설치하고, API 키 등 환경 변수를 설정한다.

In [ ]:
# 필요한 패키지 설치
%pip install -q google-adk python-dotenv

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

MODEL = "gemini-2.5-flash"

True

## 예제 1: Google 검색 도구 사용하기

첫 번째로 ADK에서 기본으로 제공하는 `Google Search` 도구를 사용하는 방법을 살펴본다. 이전 모듈에서 발생한 `ModuleNotFoundError`를 해결하기 위해 올바른 경로에서 도구를 가져온다.

In [2]:
from google.adk import Agent
# 수정됨: 올바른 도구 모듈 경로 사용
from google.adk.tools import google_search

# 검색 에이전트 생성
search_agent = Agent(
    name="search_agent",
    # 유효한 모델 이름 사용 (gemini-2.0-no-flash -> gemini-2.0-flash)
    model=MODEL,
    description="웹 검색을 통해 정보를 찾는 에이전트",
    # 한국어 지침 설정
    instruction="""
    당신은 웹 검색을 통해 최신 정보를 찾아주는 유용한 어시스턴트입니다.
    사용자의 질문에 대해 최근 이벤트나 사실 확인이 필요하면 검색 도구를 사용하세요.
    검색 결과를 바탕으로 간결하지만 충분한 정보를 제공하세요.
    가능하면 정보의 출처를 함께 언급하세요.
    """,
    # tools 리스트에 google_search 모듈 자체를 전달
    tools=[google_search]
)

### 에이전트 실행 (Runner 사용)

이전 모듈에서 학습한 대로 `InMemoryRunner`를 사용하여 에이전트를 실행한다. 에이전트는 사용자의 질문을 분석하여 검색이 필요하다고 판단되면 자동으로 `Google Search` 도구를 호출한다.

In [3]:
from google.adk.runners import InMemoryRunner

# Runner 생성
search_runner = InMemoryRunner(agent=search_agent)

# 최신 정보에 대해 질문하기 (run_debug 사용)
await search_runner.run_debug("최신 양자 컴퓨터 기술의 발전 동향에 대해 알려줘.")


 ### Created new session: debug_session_id

User > 최신 양자 컴퓨터 기술의 발전 동향에 대해 알려줘.
search_agent > 양자 컴퓨터 기술의 최신 발전 동향을 알아보기 위해 검색을 해보겠습니다.
 최신 양자 컴퓨터 기술의 발전 동향은 다음과 같습니다:

 *   **양자 컴퓨팅의 개념:** 양자 컴퓨터는 양자역학의 원리를 이용하여 정보를 처리하는 기술로, 기존 컴퓨터보다 훨씬 빠른 속도로 복잡한 계산을 수행할 수 있습니다. 0과 1의 조합을 동시에 나타내고 저장할 수 있는 큐비트(qubits)를 이용하여 전통적인 컴퓨터보다 매우 빠른 데이터 처리 속도를 갖습니다.
 *   **주요 기술 동향:**
    *   **큐비트(Qubit):** 큐비트는 양자 컴퓨터의 기본 단위로, 0과 1을 동시에 표현할 수 있는 양자의 성질인 양자 병렬성을 가집니다.
    *   **양자 우월성:** 양자 특성을 이용해 기존 기술의 한계를 뛰어넘는 성능을 실현하는 개념으로, 양자 컴퓨터가 기존 슈퍼컴퓨터보다 월등한 연산 능력을 구현하며 압도적인 성능 우위를 보이는 것을 의미합니다.
    *   **오류 감소:** 큐비트 수가 많아질수록 오류가 기하급수적으로 줄어드는 수준에 도달하고 있습니다.
    *   **하이브리드 모델:** 향후 5~10년 내에 하이브리드 모델을 통한 상용화 가능성이 크게 높아질 것으로 예상됩니다.
 *   **큐비트 구현 방식:** 양자 컴퓨터의 핵심 구성 요소인 큐비트의 구현 방식은 초전도, 이온 트랩, 광자, 중성 원자 등 여러 가지로 나뉩니다.
    *   **초전도 방식:** 금속 초전도체로 큐비트를 생성하는 방식으로 IBM, 구글 등 주요 기술 기업에서 많이 사용하며 기술적으로 가장 앞서 있습니다.
    *   **이온 트랩 방식:** 1995년 양자 컴퓨터를 위한 기술로 처음 제안되었으며, 양자 컴퓨터 구현에 필요한 기본적인 기준들을 탁월한 성능으로 충족하여 초전도체 방식과 함께 가장 실

[Event(model_version='gemini-2.0-flash', content=Content(
   parts=[
     Part(
       text="""양자 컴퓨터 기술의 최신 발전 동향을 알아보기 위해 검색을 해보겠습니다.
 """
     ),
     Part(
       text=""" 최신 양자 컴퓨터 기술의 발전 동향은 다음과 같습니다:
 
  *   **양자 컴퓨팅의 개념:** 양자 컴퓨터는 양자역학의 원리를 이용하여 정보를 처리하는 기술로, 기존 컴퓨터보다 훨씬 빠른 속도로 복잡한 계산을 수행할 수 있습니다. 0과 1의 조합을 동시에 나타내고 저장할 수 있는 큐비트(qubits)를 이용하여 전통적인 컴퓨터보다 매우 빠른 데이터 처리 속도를 갖습니다.
  *   **주요 기술 동향:**
     *   **큐비트(Qubit):** 큐비트는 양자 컴퓨터의 기본 단위로, 0과 1을 동시에 표현할 수 있는 양자의 성질인 양자 병렬성을 가집니다.
     *   **양자 우월성:** 양자 특성을 이용해 기존 기술의 한계를 뛰어넘는 성능을 실현하는 개념으로, 양자 컴퓨터가 기존 슈퍼컴퓨터보다 월등한 연산 능력을 구현하며 압도적인 성능 우위를 보이는 것을 의미합니다.
     *   **오류 감소:** 큐비트 수가 많아질수록 오류가 기하급수적으로 줄어드는 수준에 도달하고 있습니다.
     *   **하이브리드 모델:** 향후 5~10년 내에 하이브리드 모델을 통한 상용화 가능성이 크게 높아질 것으로 예상됩니다.
  *   **큐비트 구현 방식:** 양자 컴퓨터의 핵심 구성 요소인 큐비트의 구현 방식은 초전도, 이온 트랩, 광자, 중성 원자 등 여러 가지로 나뉩니다.
     *   **초전도 방식:** 금속 초전도체로 큐비트를 생성하는 방식으로 IBM, 구글 등 주요 기술 기업에서 많이 사용하며 기술적으로 가장 앞서 있습니다.
     *   **이온 트랩 방식:** 1995년 양자 컴퓨터를 위한 기술로 처음 제안되었으며, 양자 컴퓨터 

출력된 로그를 통해 에이전트가 도구를 호출하고 결과를 받아오는 과정을 확인할 수 있다.

## 예제 2: 커스텀 함수(Custom Function) 도구 만들기

이번에는 우리가 직접 Python으로 작성한 함수를 에이전트에게 도구로 제공하는 방법을 알아본다. ADK는 Python 함수의 **타입 힌트(Type Hints)**와 **닥스트링(Docstring)**을 분석하여 자동으로 도구 정의를 생성한다.

### 함수 정의하기 (중요)

함수를 도구로 사용할 때 가장 중요한 것은 **명확한 문서화**다.
- **Docstring**: 함수가 무엇을 하고, 각 인자가 어떤 의미인지 설명해야 LLM이 언제 어떻게 이 도구를 쓸지 결정할 수 있다.
- **Type Hints**: 인자와 반환값의 타입을 명시해야 ADK가 올바른 입력값을 전달할 수 있다.

In [4]:
import random

# 1. 함수 정의 (타입 힌트와 Docstring 필수)
def get_weather(location: str) -> dict:
    """
    특정 지역의 현재 날씨 정보를 가져옵니다.
    
    Args:
        location: 날씨를 알고 싶은 도시 또는 지역 이름
    
    Returns:
        온도, 날씨 상태, 습도 정보를 포함한 딕셔너리
    """
    # 실제 환경에서는 기상청 API 등을 호출하겠지만, 여기서는 모의 데이터를 생성한다.
    
    print(f"[System] '{location}'의 날씨 정보를 조회합니다...") # 호출 확인용 로그
    
    temperature = round(random.uniform(-5, 35), 1)
    conditions = random.choice(["맑음", "구름 조금", "흐림", "비", "폭풍우", "눈"])
    humidity = random.randint(30, 90)
    
    return {
        "location": location,
        "temperature": f"{temperature}°C",
        "conditions": conditions,
        "humidity": f"{humidity}%",
        "note": "이것은 시뮬레이션된 데이터입니다."
    }

### 함수 등록 및 에이전트 생성

최신 ADK 버전에서는 별도의 `FunctionTool` 클래스로 감쌀 필요 없이, `tools` 리스트에 함수 이름(`get_weather`)을 그대로 넣으면 된다. ADK가 내부적으로 이를 도구로 변환한다.

In [5]:
# 2. 에이전트 생성 (함수를 직접 도구로 등록)
weather_agent = Agent(
    name="weather_agent",
    model=MODEL,
    description="날씨 정보를 제공하는 에이전트",
    instruction="""
    당신은 친절한 날씨 안내원입니다.
    사용자가 특정 지역의 날씨를 물어보면 'get_weather' 도구를 사용하여 정보를 확인하세요.
    
    제공받은 날씨 정보를 바탕으로 대화하듯이 자연스럽게 답변하세요.
    사용자가 지역을 말하지 않았다면, 어느 지역인지 물어보세요.
    """,
    tools=[get_weather]
)

# 3. Runner 생성
weather_runner = InMemoryRunner(agent=weather_agent)

### 커스텀 도구 테스트

서울의 날씨를 물어보면 에이전트가 `get_weather` 함수를 실행하는지 확인해 본다.

In [6]:
# 날씨 정보 요청
await weather_runner.run_debug("지금 서울 날씨 어때?")


 ### Created new session: debug_session_id

User > 지금 서울 날씨 어때?


[System] '서울'의 날씨 정보를 조회합니다...
weather_agent > 서울은 현재 비가 내리고 있고, 기온은 -1.6°C입니다. 습도는 63%입니다.


[Event(model_version='gemini-2.0-flash', content=Content(
   parts=[
     Part(
       function_call=FunctionCall(
         args={
           'location': '서울'
         },
         id='adk-fc49f212-9d1d-40af-a773-66e3e72737f4',
         name='get_weather'
       )
     ),
   ],
   role='model'
 ), grounding_metadata=None, partial=None, turn_complete=None, finish_reason=<FinishReason.STOP: 'STOP'>, error_code=None, error_message=None, interrupted=None, custom_metadata=None, usage_metadata=GenerateContentResponseUsageMetadata(
   candidates_token_count=5,
   candidates_tokens_details=[
     ModalityTokenCount(
       modality=<MediaModality.TEXT: 'TEXT'>,
       token_count=5
     ),
   ],
   prompt_token_count=207,
   prompt_tokens_details=[
     ModalityTokenCount(
       modality=<MediaModality.TEXT: 'TEXT'>,
       token_count=207
     ),
   ],
   total_token_count=212
 ), live_session_resumption_update=None, input_transcription=None, output_transcription=None, avg_logprobs=-0.0023163

In [7]:
## 예제 3: 멀티 도구(Multi-Tool) 에이전트

가장 강력한 기능은 하나의 에이전트가 여러 도구를 가지고 상황에 맞게 골라 사용하는 것이다. `Google Search`와 `get_weather`를 모두 가진 에이전트를 만들어 본다.


 ### Continue session: debug_session_id

User > 부산은 어때?
[System] '부산'의 날씨 정보를 조회합니다...
weather_agent > 부산은 현재 비가 내리고 있고, 기온은 8.1°C입니다. 습도는 45%입니다. 서울보다 따뜻하네요.


[Event(model_version='gemini-2.0-flash', content=Content(
   parts=[
     Part(
       function_call=FunctionCall(
         args={
           'location': '부산'
         },
         id='adk-f2605362-57ef-4a95-a670-853d40633257',
         name='get_weather'
       )
     ),
   ],
   role='model'
 ), grounding_metadata=None, partial=None, turn_complete=None, finish_reason=<FinishReason.STOP: 'STOP'>, error_code=None, error_message=None, interrupted=None, custom_metadata=None, usage_metadata=GenerateContentResponseUsageMetadata(
   candidates_token_count=6,
   candidates_tokens_details=[
     ModalityTokenCount(
       modality=<MediaModality.TEXT: 'TEXT'>,
       token_count=6
     ),
   ],
   prompt_token_count=278,
   prompt_tokens_details=[
     ModalityTokenCount(
       modality=<MediaModality.TEXT: 'TEXT'>,
       token_count=278
     ),
   ],
   total_token_count=284
 ), live_session_resumption_update=None, input_transcription=None, output_transcription=None, avg_logprobs=-4.8093070

## 도구 정의하기 (Custom Functions)

에이전트가 사용할 두 가지 도구를 정의한다. 
1. `get_weather`: 날씨 정보를 제공하는 도구
2. `mock_search`: 웹 검색을 시뮬레이션하는 도구 (API 호환성 문제를 피하기 위해 내장 도구 대신 사용)

**중요**: 함수 도구를 만들 때는 **Docstring(설명)**과 **Type Hint(타입)**를 명확하게 작성해야 에이전트가 언제 이 도구를 쓸지 올바르게 판단할 수 있다.

## 도구 정의하기 (Custom Functions)

에이전트가 사용할 두 가지 도구를 정의한다. 
1. `get_weather`: 날씨 정보를 제공하는 도구
2. `mock_search`: 웹 검색을 시뮬레이션하는 도구 (API 호환성 문제를 피하기 위해 내장 도구 대신 사용)

**중요**: 함수 도구를 만들 때는 **Docstring(설명)**과 **Type Hint(타입)**를 명확하게 작성해야 에이전트가 언제 이 도구를 쓸지 올바르게 판단할 수 있다.

In [13]:
import random

# 1. 날씨 정보 도구
def get_weather(location: str) -> dict:
    """
    특정 지역의 현재 날씨 정보를 가져옵니다.
    
    Args:
        location: 날씨를 알고 싶은 도시 또는 지역 이름
    
    Returns:
        온도, 날씨 상태, 습도 정보를 포함한 딕셔너리
    """
    # 에이전트가 도구를 호출했는지 확인하기 위한 로그
    print(f"\n[System] 🌦️ 날씨 도구 호출됨: '{location}'")
    
    # 시뮬레이션 데이터 반환
    return {
        "location": location,
        "temperature": "24°C",
        "condition": "맑음",
        "humidity": "45%"
    }

# 2. 검색 시뮬레이션 도구
def mock_search(query: str) -> str:
    """
    웹 검색을 수행하여 최신 정보나 사실 관계를 찾습니다.
    관광지 추천, 뉴스, 일반적인 지식 검색에 사용하세요.
    
    Args:
        query: 검색하고 싶은 키워드나 질문 내용
        
    Returns:
        검색 결과 텍스트 요약
    """
    print(f"\n[System] 🔍 검색 도구 호출됨: '{query}'")
    
    if "관광" in query and "파리" in query:
        return "파리 추천 관광지: 1. 에펠탑 (상징적인 랜드마크), 2. 루브르 박물관 (예술 작품 감상), 3. 몽마르뜨 언덕 (예술가의 거리)"
    elif "양자 컴퓨터" in query:
        return "최근 양자 컴퓨터는 오류 수정 기술에서 큰 진전을 이루었으며, 상용화를 위한 단계에 진입하고 있습니다."
    else:
        return f"'{query}'에 대한 검색 결과: 관련 정보를 찾았습니다."


In [14]:
## 도구 선택 테스트 (Routing)

에이전트가 질문의 의도에 따라 도구를 올바르게 바꿔가며 사용하는지 테스트해 본다.

1. **날씨 질문**: `get_weather` 호출 예상
2. **관광지 질문**: `mock_search` 호출 예상

## 도구 선택 테스트 (Routing)

에이전트가 질문의 의도에 따라 도구를 올바르게 바꿔가며 사용하는지 테스트한다.

1. **날씨 질문**: `get_weather` 호출 예상
2. **관광지 질문**: `mock_search` 호출 예상

In [16]:
print("=== 테스트 1: 날씨 질문 (날씨 도구 호출 예상) ===")
await runner.run_debug("지금 파리 날씨는 어때?")

=== 테스트 1: 날씨 질문 (날씨 도구 호출 예상) ===

 ### Created new session: debug_session_id

User > 지금 파리 날씨는 어때?

[System] 🌦️ 날씨 도구 호출됨: 'Paris'
multi_tool_agent > 현재 파리는 맑고, 온도는 24°C이며 습도는 45%입니다.


[Event(model_version='gemini-2.0-flash', content=Content(
   parts=[
     Part(
       function_call=FunctionCall(
         args={
           'location': 'Paris'
         },
         id='adk-2c33e254-43ef-41a9-b84d-78ff4e98bd73',
         name='get_weather'
       )
     ),
   ],
   role='model'
 ), grounding_metadata=None, partial=None, turn_complete=None, finish_reason=<FinishReason.STOP: 'STOP'>, error_code=None, error_message=None, interrupted=None, custom_metadata=None, usage_metadata=GenerateContentResponseUsageMetadata(
   candidates_token_count=5,
   candidates_tokens_details=[
     ModalityTokenCount(
       modality=<MediaModality.TEXT: 'TEXT'>,
       token_count=5
     ),
   ],
   prompt_token_count=324,
   prompt_tokens_details=[
     ModalityTokenCount(
       modality=<MediaModality.TEXT: 'TEXT'>,
       token_count=324
     ),
   ],
   total_token_count=329
 ), live_session_resumption_update=None, input_transcription=None, output_transcription=None, avg_logprobs=-0.0711

## 정리 및 모범 사례

이번 모듈에서는 ADK 에이전트의 핵심 기능인 **도구(Tools)**를 다뤘다.

1. **함수 도구(Function Calling)**: Python 함수를 그대로 에이전트의 도구로 등록할 수 있다. 이때 Docstring이 에이전트의 판단 기준이 된다.
2. **도구 선택(Routing)**: 에이전트는 사용자의 질문을 분석하여 여러 도구 중 가장 적합한 것을 스스로 선택한다.
3. **호환성 고려**: 현재 Gemini 모델에서는 내장 도구(Grounding)와 커스텀 함수를 섞어 쓰는 것에 제약이 있을 수 있으므로, 필요하다면 커스텀 함수로 통일하여 구현하는 것이 안정적이다.

### 도구 개발 체크리스트
1. **단일 책임 원칙**: 하나의 도구는 하나의 명확한 기능만 수행하도록 작게 만든다.
2. **명확한 이름과 설명**: 함수 이름은 직관적으로(예: `get_weather`), Docstring은 상세하게 작성한다. LLM은 이 설명을 읽고 도구 사용 여부를 결정한다.
3. **타입 힌트 준수**: `str`, `int`, `dict` 등 타입을 명시해야 오류를 줄일 수 있다.
4. **구조화된 반환값**: 텍스트보다는 JSON(dict) 형태로 반환하는 것이 에이전트가 정보를 해석하기 좋다.

다음 모듈에서는 에이전트 간에 협업을 할 수 있는 **멀티 에이전트 시스템**에 대해 알아볼 것이다.